In [ ]:
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt

# File path
file_path = "../data/eric_ingressgw_udm_traffic_cpu_time_series_lk5nv.csv"

# Load the CSV file into a Pandas DataFrame
time_series_cpu_lk5nv = pd.read_csv(file_path)

# Display the first few rows of the data
print(time_series_cpu_lk5nv.head())


time_series_cpu_lk5nv = pd.read_csv(file_path, header=None, names=["Time Start", "Cluster", "CPU Utilization (%)"])
print(time_series_cpu_lk5nv.info())
print(time_series_cpu_lk5nv.describe())

In [ ]:
# Ensure the DataFrame has the correct columns for Prophet
time_series_cpu_lk5nv = time_series_cpu_lk5nv[["Time Start", "CPU Utilization (%)"]].copy()
time_series_cpu_lk5nv.columns = ["ds", "y"]  # Rename columns for Prophet

# Ensure datetime format for 'ds'
time_series_cpu_lk5nv["ds"] = pd.to_datetime(time_series_cpu_lk5nv["ds"], errors='coerce')

# Ensure 'y' is numeric
time_series_cpu_lk5nv["y"] = pd.to_numeric(time_series_cpu_lk5nv["y"], errors='coerce')

# Drop rows with missing values in 'ds' or 'y'
time_series_cpu_lk5nv = time_series_cpu_lk5nv.dropna(subset=["ds", "y"])

# Display the cleaned DataFrame for validation
print(time_series_cpu_lk5nv.head())

# Initialize and fit the Prophet model
model = Prophet()
model.fit(time_series_cpu_lk5nv)





Forecast next 60 minutes: 

In [ ]:
# Create a future dataframe for the next 10 minutes
future = model.make_future_dataframe(periods=10, freq='T')
forecast = model.predict(future)

# Plot the forecast
fig = model.plot(forecast)
plt.title("10-Minute Forecast of CPU Utilization for 'capo-28037-pool1-lk5nv'")
plt.xlabel("Time")
plt.ylabel("CPU Utilization (%)")
plt.show()

# Display forecast results for the next 10 minutes
print(forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail(10))
